In [30]:
import sqlite3
import pandas as pd
import shutil
import os.path

path = 'C:/Users/gdoku/YandexDisk/Компания/07 Учётные регистры и первичные документы/'
file = path + 'accounting.db'

check_file = os.path.exists(path + 'accounting_old.db') # True
if not check_file:
    shutil.copyfile(file, path + 'accounting_old.db')

conn = sqlite3.connect(file)
c = conn.cursor()

tables = c.execute(
    "SELECT `name`, `sql` FROM `sqlite_master` WHERE type='table'").fetchall()

# Удаляем старые таблицы
# for table in tables:
#     if 'temp' not in table[0]:
#         c.execute("DROP TABLE {}".format(table[0]))

# Создаём таблицы
c.execute('CREATE TABLE IF NOT EXISTS sc_types (\
              sc_type_id INTEGER,\
              sc_type TEXT NOT NULL UNIQUE,\
              doc INTEGER NOT NULL CHECK (doc=0 OR doc=1) DEFAULT 0, \
              PRIMARY KEY (sc_type_id),\
              UNIQUE (sc_type_id, doc))'
          )
c.execute('CREATE TABLE IF NOT EXISTS accounts ( \
              account INTEGER, \
              account_name TEXT NOT NULL, \
              sc_type_id_1 INTEGER REFERENCES sc_types (sc_type_id), \
              sc_type_id_2 INTEGER REFERENCES sc_types (sc_type_id), \
              PRIMARY KEY (account)) WITHOUT ROWID'
          )
c.execute('CREATE TABLE IF NOT EXISTS document_types ( \
              document_type_id INTEGER, \
              document_type TEXT NOT NULL UNIQUE, \
              payment INTEGER UNIQUE CHECK (payment ISNULL OR payment=1), \
              PRIMARY KEY (document_type_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS documents ( \
              document_id INTEGER, \
              document_type_id INTEGER NOT NULL REFERENCES document_types (document_type_id),\
              document_date TEXT NOT NULL CHECK (date(document_date) NOTNULL), \
              document_number TEXT, \
              comment TEXT, \
              PRIMARY KEY (document_id),\
              CHECK (document_number NOTNULL OR comment NOTNULL))'
          )
c.execute('CREATE TABLE IF NOT EXISTS subcounts ( \
              subcount_id INTEGER, \
              sc_type_id INTEGER NOT NULL REFERENCES sc_types (sc_type_id), \
              sc_name TEXT, \
              document_id INTEGER REFERENCES documents (document_id), \
              archive INTEGER NOT NULL CHECK (archive=0 OR archive=1) DEFAULT 0, \
              doc INTEGERG NOT NULL CHECK (doc=0 OR doc=1) DEFAULT 0, \
              PRIMARY KEY (subcount_id),\
              FOREIGN KEY (sc_type_id, doc) REFERENCES sc_types (sc_type_id, doc),\
              CHECK ((doc=1 AND document_id NOTNULL) OR (doc=0 AND document_id ISNULL)))'
          )
c.execute('CREATE TABLE IF NOT EXISTS operations_types ( \
            operation_type_id INTEGER,\
            operation_type TEXT NOT NULL UNIQUE, \
            PRIMARY KEY (operation_type_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS operations ( \
            operation_id INTEGER,\
            value_date TEXT CHECK (date(value_date) IS NOT NULL),\
            creation_date TEXT NOT NULL CHECK (datetime(creation_date) IS NOT NULL) DEFAULT CURRENT_TIMESTAMP, \
            document_id INTEGER NOT NULL REFERENCES documents (document_id), \
            operation_type_id INTEGER NOT NULL REFERENCES operations_types (operation_type_id), \
            registered INTEGER NOT NULL CHECK (registered=0 OR registered=1) DEFAULT 0, \
            PRIMARY KEY (operation_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS transactions ( \
            trans_id INTEGER,\
            operation_id INTEGER NOT NULL REFERENCES operations (operation_id), \
            db_account INTEGER NOT NULL REFERENCES accounts (account_id), \
            db_sc_1_id INTEGER REFERENCES subcounts (subcount_id), \
            db_sc_2_id INTEGER REFERENCES subcounts (subcount_id), \
            cr_account INTEGER NOT NULL REFERENCES accounts (account_id), \
            cr_sc_1_id INTEGER REFERENCES subcounts (subcount_id), \
            cr_sc_2_id INTEGER REFERENCES subcounts (subcount_id), \
            value REAL NOT NULL, \
            amount REAL, \
            PRIMARY KEY (trans_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS payments ( \
            payment_id INTEGER NOT NULL REFERENCES documents (document_id), \
            value REAL NOT NULL, \
            description TEXT NOT NULL, \
            bik INTEGER NOT NULL, \
            account TEXT NOT NULL, \
            reciver TEXT NOT NULL, \
            PRIMARY KEY (payment_id))'
          )
c.execute('CREATE VIEW IF NOT EXISTS payments_list AS\
            SELECT  documents.document_id, \
                    documents.document_date,\
                    documents.document_number,\
                    payments.value,\
                    payments.description,\
                    payments.bik,\
                    payments.account,\
                    payments.reciver\
            FROM (documents LEFT JOIN payments ON documents.document_id = payments.payment_id)\
                    INNER JOIN document_types ON documents.document_type_id = document_types.document_type_id\
            WHERE document_types.payment=1\
          ')
c.execute('\
    CREATE VIEW IF NOT EXISTS subcount_list AS\
    SELECT \
        subcounts.subcount_id, \
        subcounts.sc_type_id, \
        sc_types.sc_type, \
        coalesce(subcounts.sc_name , document_types.document_type) \
        || coalesce(\' № \'|| documents.document_number, \'\') \
        || coalesce(\' от \'  || documents.document_date, \'\') \
        || coalesce(\'; \' || documents.comment, \'\') AS subcount, \
        subcounts.doc \
    FROM \
        subcounts INNER JOIN sc_types ON subcounts.sc_type_id = sc_types.sc_type_id LEFT JOIN \
        documents ON subcounts.document_id = documents.document_id LEFT JOIN \
        document_types ON documents.document_type_id = document_types.document_type_id \
        ')
c.execute('\
CREATE VIEW IF NOT EXISTS transaction_log AS \
SELECT \
	transactions.trans_id, \
	operations.registered, \
	operations.value_date, \
	document_types.document_type \
        || coalesce(\' № \'|| documents.document_number, \'\') \
        || coalesce(\' от \'  || documents.document_date, \'\') \
        || coalesce(\'; \' || documents.comment, \'\') AS document, \
	operations_types.operation_type, \
	transactions.db_account AS account, \
	coalesce(subcount_list.subcount, accounts.account_name) AS subcount, \
	transactions.value AS db, \
	Null AS cr \
FROM  \
	transactions INNER JOIN \
	operations ON transactions.operation_id = operations.operation_id LEFT JOIN \
	operations_types ON operations.operation_type_id = operations_types.operation_type_id LEFT JOIN \
	subcount_list ON transactions.db_sc_1_id = subcount_list.subcount_id LEFT JOIN \
	documents ON operations.document_id = documents.document_id LEFT JOIN \
	document_types ON documents.document_type_id = document_types.document_type_id INNER JOIN \
	accounts ON transactions.db_account = accounts.account \
UNION \
SELECT \
	transactions.trans_id, \
	operations.registered, \
	operations.value_date, \
	document_types.document_type \
		|| coalesce(\' № \'|| documents.document_number, \'\') \
		|| coalesce(\' от \'  || documents.document_date, \'\') \
		|| coalesce(\'; \' || documents.comment, \'\') AS document, \
	operations_types.operation_type, \
	transactions.cr_account AS account, \
	coalesce(subcount_list.subcount, accounts.account_name) AS subcount, \
	Null AS db, \
	transactions.value AS cr \
FROM  \
	transactions INNER JOIN \
	operations ON transactions.operation_id = operations.operation_id LEFT JOIN \
	operations_types ON operations.operation_type_id = operations_types.operation_type_id LEFT JOIN \
	subcount_list ON transactions.cr_sc_1_id = subcount_list.subcount_id LEFT JOIN \
	documents ON operations.document_id = documents.document_id LEFT JOIN \
	document_types ON documents.document_type_id = document_types.document_type_id INNER JOIN \
	accounts ON transactions.cr_account = accounts.account \
        ')

conn.commit()
conn.close()

In [13]:
conn = sqlite3.connect(file)
c = conn.cursor()
data = c.execute('SELECT group_concat(sc_type_id) FILTER (WHERE doc=1) FROM sc_types').fetchall()
# data = c.execute(
#     "SELECT `name`, `sql`\
#   FROM `sqlite_master` \
#   WHERE type='table'").fetchall()
#data = pd.DataFrame(data)
# for table in data:
#     if 'temp' not in table[0]:
#         print(table[0])
#    c.execute("DROP TABLE {}".format(table[0]+'_temp'))
conn.close()
data

[('3,4',)]

In [94]:
from tkinter import *

root = Tk()
root.title("Hello World!")
root.geometry('300x40')

def button_clicked():
    print("Hello World!")

def close():
    root.destroy()
    root.quit()

button = Button(root, text="Press Me", command=button_clicked)
button.pack(fill=BOTH)

root.protocol('WM_DELETE_WINDOW', close)

root.mainloop()

Hello World!
Hello World!
Hello World!
